# Webscrapes vlr.gg to get stats of the players before their match and stats of the teams head to head

## Imports

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from io import StringIO
from typing import Optional, Union
from functools import lru_cache

#### Constants for webscraping

In [3]:
folderpath = "G:\\School\\PersonalSVN\\S24\\CS372\\FinalProject\\"

# These are used to find different sections. they are extracted becasue the sometimes change
matches_info_block = "wf-module-item match-item"
matches_info_vs_block = "match-item-vs"
matches_info_team_name = "match-item-vs-team-name"
matches_info_team_score = "match-item-vs-team-score"
matches_info_series_name = "match-item-event-series"

stats_series_class = "wf-tag-btn noselect series-id-"
stats_series_id = "data-subseries-id"

abbreviations_td_class = "mod-player"
abbreviations_class = "ge-text-light"

america_kickoff_event_link = "https://www.vlr.gg/event/1660/champions-tour-2024-americas-kickoff"
# excluding 1926 because bad data
event_codes = [1921, 1925, 1924, 1923, 1657, 1658, 1659, 1660, 1664, 1494, 1189, 1190, 1191, 1188, 
               1453, 1953, 1653, 1653, 1875, 1763, 1750]


#### Gets links and the Soup object for the website

In [4]:
def get_links(event_link: str) -> tuple[str, str]:
  index = event_link.find("/event/") + 7
  stats_link = event_link[:index] + "stats/" + event_link[index:]
  
  matches_link = event_link[:index] + "matches/" + event_link[index:] + "?series_id=all"
  
  return stats_link, matches_link
  
def get_soups(stats_link: str, match_link: str) -> tuple[BeautifulSoup, BeautifulSoup]:
  # Load stats
  statsPage = requests.get(stats_link)
  statsSoup = BeautifulSoup(statsPage.content, 'html.parser')

  # Load matches
  matchesPage = requests.get(match_link)
  matchesSoup = BeautifulSoup(matchesPage.content, 'html.parser')
  
  return statsSoup, matchesSoup

### Gets all matches in the lists of matches

In [5]:
# Get all teams, winners, and series
def get_matches(matchesSoup: BeautifulSoup) -> pd.DataFrame:
  # All match info blocks
  match_info_blocks = matchesSoup.find_all("a", class_=lambda x: x and x.startswith(matches_info_block))
  matches = pd.DataFrame(columns=["date", "team1", "team2", "winner", "series", "code"])
  for div in match_info_blocks:
    # get the parent div
    parent_div = div.parent
    # get the date from the dive above the parent div
    date = parent_div.find_previous("div", class_="wf-label mod-large").text.strip()
    # drops the day of the week
    date = ", ".join(date.split(", ")[1:])
    
    # get the two divs in the div "match-item-vs"
    vs = div.find("div", class_=matches_info_vs_block)
    # get the two team names
    teamNames = vs.find_all("div", class_=matches_info_team_name)
    
    team1 = teamNames[0].text.strip()
    team2 = teamNames[1].text.strip()
    
    # get the two scores (starts with "match-item-vs-team-score")
    scores = vs.find_all("div", class_=lambda x: x and x.startswith(matches_info_team_score))
    score1 = scores[0].text.strip()
    score2 = scores[1].text.strip()
    
    # get the series name
    series = div.find("div", class_=lambda x: x and x.startswith(matches_info_series_name)).text.strip()
    
    # Gets the link (for finding abbreviation later)
    code = div["href"].split("/")[1]
    
    # Gets the winner (1 if team1, 2 if team2)
    winner = 1 if score1 > score2 else 2
    
    # add the df
    matches = pd.concat([matches, pd.DataFrame({"date": [date], "team1": [team1], "team2": [team2], "winner": [winner], "series": [series], "code": [code]})])
  
  return matches

### Get the list of series and their codes for getting stats

In [6]:
# Gets all the different series and codes
def get_series(statsSoup: BeautifulSoup, matches: pd.Series) -> pd.DataFrame:
  # gets all the different cutoffs to exclude
  # this is the class of the divs that contain the character names
  divs = statsSoup.find_all("div", class_=lambda x: x and x.startswith(stats_series_class))

  # Put them in a df of the content of the div labeled name and the data-subseries-id
  series = pd.DataFrame(columns=["name", "id"])
  for div in divs:
    name = div.text
    # remove the padding from the name
    name = name.strip()
    id = div[stats_series_id]
    # use concat to add the new row to the df
    series = pd.concat([series, pd.DataFrame({"name": [name], "id": [id]})], ignore_index=True)
  
  series["name"] = pd.Categorical(series["name"], categories=matches.unique(), ordered=True)
  series = series.sort_values("name")
  return series

### Get the soups for all the player stats

In [7]:
def get_different_cutoffs_soups(stats_link: str, series: pd.DataFrame) -> dict[str, BeautifulSoup]:
  # Get soups of all different cutoffs on match page
  # The link would stats_link + "?exclude=" + the id of all matches below the current one in the matches df joined by a .
  match_soups = {}
  for i in range(len(series)):
    cutoff = ".".join(series["id"].iloc[i:])
    match_soups[series["name"].iloc[i]] = BeautifulSoup(requests.get(stats_link + "?exclude=" + cutoff).content, 'html.parser')
    #match_soups[series["name"].iloc[i]] = stats_link + "?exclude=" + cutoff
  
  return match_soups

### Gets the player stats from the soup

In [8]:
def get_stats(soup: BeautifulSoup):
  # Get the table with class wf-table mod-stats mod-scroll
  table = soup.find("table", {"class": "wf-table mod-stats mod-scroll"})

  # Get the different columns
  columns = table.find("thead").find_all("th")
  
  # Get the table body
  table_body = table.find("tbody")

  # Get all the rows
  rows = table_body.find_all("tr")

  # Put into pandas dataframe
  data = []
  for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]
    data.append(cols)

  df = pd.DataFrame(data, columns=[column.text for column in columns])
  
  if (len(df) == 0):
    # no rows so just add the Team column
    df["Team"] = ""
  else:
    # seperate the player and team at the \n in the player column
    df[["Player", "Team"]] = df["Player"].str.split("\n", expand=True)
  
  # make team column the second column
  df = df[["Player", "Team"] + [col for col in df.columns if col not in ["Player", "Team"]]]

  # remove agent column
  df = df.drop("Agents", axis=1)
  return df

def get_series_stats(cutoff_soups: dict[str, BeautifulSoup]) -> pd.DataFrame:
  stats = {}
  for key in cutoff_soups:
    stat = get_stats(cutoff_soups[key])
    if (len(stat) > 0):
      stats[key] = stat
  return stats

### Clean the player statistics

Changes empty string to NaN, sets strings to floats, removes % signs, and separates clutches

In [48]:
def clean_stat(stat: pd.DataFrame) -> pd.DataFrame:
  # make all row types strings
  stat = stat.astype(str)
  # If a value is an empty string, replace it with NaN
  stat = stat.replace("", float("NaN"))
  
  first_valid_row = stat.dropna().index[0]

  # Get the first row in a column and check if it is a percentage. if so turn into float
  for column in stat.columns[2:]:
    if "%" in stat[column].loc[first_valid_row]:
      stat[column] = stat[column].str.replace("%", "").astype(float)
  
  # If it is a number, turn into float
  for column in stat.columns[2:]:
    # continue if not str
    if stat[column] is not str:
      continue
    if stat[column].loc[first_valid_row].replace(".", "", 1).isdigit():
      stat[column] = stat[column].astype(float)
      
      
  # Split CL into success and attempts
  if "CL" in stat.columns:
    cols = stat["CL"].str.split("/", expand=True)
    # Insert the two new columns at the CL column
    CL_index = stat.columns.get_loc("CL") + 1
    stat.insert(CL_index, "CL Success", cols[0].astype(float))
    stat.insert(CL_index + 1, "CL Attempts", cols[1].astype(float))
    stat = stat.drop("CL", axis=1)
  
  return stat

# Runs clean_stat on all stats in the dict
def clean_stats(stats: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
  for key in stats:
    stats[key] = clean_stat(stats[key])
  return stats

### Get team abbreviations for matching player stats to matches

In [49]:
# Get team abbreviations for matching player stats to matches
def turn_name_into_abbreviations(matches: pd.DataFrame) -> pd.DataFrame:
  # Go through all matches
  abbreviations = {}
  for index, match_ in matches.iterrows():
    # If both teams are already in the abbreviations, continue
    if match_["team1"] in abbreviations and match_["team2"] in abbreviations:
      continue
    
    # Open the link
    page = requests.get("https://www.vlr.gg/" + match_["code"])
    match_soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get all td with class mod-player
    tds = match_soup.find_all("td", class_=abbreviations_td_class)
    
    team1_td = tds[-6]
    team2_td = tds[-1]
    
    #team1_abbreviation = team_1_td[0][1][1].text
    team1_abbreviation = team1_td.find('div', class_=abbreviations_class).text.strip()
    
    team2_abbreviation = team2_td.find('div', class_=abbreviations_class).text.strip()
    
    # Add the abbreviations to the dict
    abbreviations[match_["team1"]] = team1_abbreviation
    abbreviations[match_["team2"]] = team2_abbreviation
  
  # Convert old names to abbreviations
  matches["team1_abrev"] = matches["team1"].map(abbreviations)
  matches["team2_abrev"] = matches["team2"].map(abbreviations)
  
  # Remove the link column
  #matches = matches.drop("link", axis=1)
  
  return matches


### Adding team stats to match lists

In [50]:
# Gets stats of only players on team
def get_team_stats(team: str, stats: pd.DataFrame) -> pd.DataFrame:
  return stats[stats["Team"] == team]

# Adds teams stat columns to matches
def add_team_stats_to_matches(matches: pd.DataFrame, stats: dict[str, pd.DataFrame]) -> pd.DataFrame:
  matches.reset_index(drop=True, inplace=True)
  
  # for each map
  for index, match_ in matches.iterrows():
    # get series name
    series = match_["series"]
    # if the series is not in the stats remove the row
    if series not in stats:
      matches = matches.drop(index)
      continue
    # get the stats for the series
    stat = stats[series]
    # get the team stats
    team1_stats = get_team_stats(match_["team1_abrev"], stat)
    team2_stats = get_team_stats(match_["team2_abrev"], stat)
    
    # If either team is not in the stats remove the row
    if len(team1_stats) == 0 or len(team2_stats) == 0:
      matches = matches.drop(index)
      continue
    
    # add the stats to the matches df
    matches.at[index, "team1_stats"] = team1_stats.to_csv(index=False)
    matches.at[index, "team2_stats"] = team2_stats.to_csv(index=False)

  return matches

# Function for getting stats from a link or a list of codes

In [51]:
@lru_cache(maxsize=None)
def get_event_stats(event_link: str) -> Optional[pd.DataFrame]:
  print("Starting", event_link)
  if type(event_link) == list:
    if len(event_link) == 0:
      return None
    # Get first link
    event_stats = get_event_stats(event_link[0])
    print("Done with", event_link[0])
    print("Length of event_stats:", len(event_stats))
    
    # Add all other links
    for link in event_link[1:]:
      event_stats = pd.concat([event_stats, get_event_stats(link)], ignore_index=True)
      print("Done with", link)
      print("Length of event_stats:", len(event_stats))
    return event_stats
  
  # Get the soups
  stats_link, matches_link = get_links(event_link)
  stats_soup, matches_soup = get_soups(stats_link, matches_link)
  
  # Get matches and series
  matches = get_matches(matches_soup)
  series = get_series(stats_soup, matches["series"])

  # Getting player stats
  cutoff_soups = get_different_cutoffs_soups(stats_link, series)
  stats = get_series_stats(cutoff_soups)
  stats = clean_stats(stats)

  # Add player stats to matches
  matches = turn_name_into_abbreviations(matches)
  return add_team_stats_to_matches(matches, stats)


### Getting matches from given link

# [link to player statistics](https://www.vlr.gg/event/stats/2004/champions-tour-2024-americas-stage-1)

In [52]:
event_stats = get_event_stats(america_kickoff_event_link)
display(event_stats)
display(pd.read_csv(StringIO(event_stats["team1_stats"].iloc[0])))
display(pd.read_csv(StringIO(event_stats["team2_stats"].iloc[0])))

Starting https://www.vlr.gg/event/1660/champions-tour-2024-americas-kickoff


,date,team1,team2,winner,series,code,team1_abrev,team2_abrev,team1_stats,team2_stats
5,"July 21, 2023",ZETA DIVISION,Team Secret,1,Upper Final,237271,ZETA,TS,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP..."
6,"July 21, 2023",Rex Regum Qeon,Gen.G,2,Lower Round 1,237272,RRQ,GEN,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP..."
7,"July 22, 2023",Team Secret,Gen.G,1,Lower Final,237273,TS,GEN,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP..."
8,"July 23, 2023",ZETA DIVISION,Team Secret,1,Grand Final,237274,ZETA,TS,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP..."


,Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,...,HS%,CL%,CL Success,CL Attempts,KMax,K,D,A,FK,FD
0,Dep,ZETA,76,1.00,215.3,1.20,76.0,159.2,0.87,0.22,...,32.0,18.0,2.0,11.0,42,66,55,17,8,6
1,TENNN,ZETA,76,0.94,221.0,1.07,74.0,144.2,0.82,0.13,...,22.0,NaN,0.0,5.0,31,62,58,10,15,14
2,Laz,ZETA,76,0.90,165.0,0.81,70.0,114.2,0.57,0.29,...,23.0,17.0,2.0,12.0,21,43,53,22,4,6
3,SugarZ3ro,ZETA,76,0.88,165.3,0.98,78.0,122.7,0.68,0.58,...,28.0,13.0,1.0,8.0,34,52,53,44,5,8
4,crow,ZETA,76,0.85,180.3,0.76,76.0,121.6,0.58,0.55,...,24.0,10.0,1.0,10.0,23,44,58,42,8,2


,Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,...,HS%,CL%,CL Success,CL Attempts,KMax,K,D,A,FK,FD
0,invy,TS,40,1.37,223.5,1.55,85.0,138.8,0.85,0.60,...,36.0,50.0,1.0,2.0,18,34,22,24,6,2
1,DubsteP,TS,40,1.34,227.0,1.61,80.0,160.6,0.93,0.10,...,40.0,25.0,2.0,8.0,19,37,23,4,4,2
2,Jremy,TS,40,1.33,261.0,1.52,78.0,163.8,0.95,0.35,...,17.0,20.0,1.0,5.0,23,38,25,14,9,9
3,BORKUM,TS,40,1.27,207.0,1.36,85.0,139.2,0.75,0.65,...,27.0,33.0,1.0,3.0,16,30,22,26,3,1
4,JessieVash,TS,40,0.95,182.5,0.85,78.0,123.1,0.58,0.43,...,17.0,100.0,1.0,1.0,12,23,27,17,2,2


### Saving the data

In [ ]:
event_stats.to_csv(folderpath + "event_stats.csv", index=False)

### Getting all stats in list of links

In [ ]:
def code_to_link(code: int) -> str:
  return f"https://www.vlr.gg/event/{code}"

def get_many_event_stats(event_codes: list[str]) -> Optional[pd.DataFrame]:
  event_links = [code_to_link(code) for code in event_codes]
  if len(event_links) == 0:
    return None
  # Get first link
  event_stats = get_event_stats(event_links[0])
  # add event code column
  event_stats["event_code"] = event_codes[0]
  print("Done with", event_links[0])
  print("Length of event_stats:", len(event_stats))

  # Add all other links
  for link, code in zip(event_links[1:], event_codes[1:]):
    stats = get_event_stats(link)
    stats["event_code"] = code
    event_stats = pd.concat([event_stats, stats], ignore_index=True)
    print("Done with", link)
    print("Length of event_stats:", len(event_stats))
  
  return event_stats


In [ ]:
stats_2023 = get_many_event_stats(event_codes)
display(stats_2023.shape)
display(stats_2023.head())
display(pd.read_csv(StringIO(stats_2023["team1_stats"].iloc[0])))

Done with https://www.vlr.gg/event/1921
Length of event_stats: 12
Done with https://www.vlr.gg/event/1925
Length of event_stats: 25
Done with https://www.vlr.gg/event/1924
Length of event_stats: 38
Done with https://www.vlr.gg/event/1923
Length of event_stats: 51
Done with https://www.vlr.gg/event/1657
Length of event_stats: 77
Done with https://www.vlr.gg/event/1658
Length of event_stats: 81
Done with https://www.vlr.gg/event/1659
Length of event_stats: 85
Done with https://www.vlr.gg/event/1660
Length of event_stats: 89
Done with https://www.vlr.gg/event/1664
Length of event_stats: 127
Done with https://www.vlr.gg/event/1494
Length of event_stats: 143
Done with https://www.vlr.gg/event/1189
Length of event_stats: 193
Done with https://www.vlr.gg/event/1190
Length of event_stats: 238
Done with https://www.vlr.gg/event/1191
Length of event_stats: 288
Done with https://www.vlr.gg/event/1188
Length of event_stats: 303
Done with https://www.vlr.gg/event/1453
Length of event_stats: 307
Don

(376, 11)

,date,team1,team2,winner,series,code,team1_abrev,team2_abrev,team1_stats,team2_stats,event_code
0,"March 16, 2024",Sentinels,Karmine Corp,1,Round 2,312798,SEN,KC,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921
1,"March 16, 2024",EDward Gaming,Gen.G,2,Round 2,312797,EDG,GEN,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921
2,"March 17, 2024",FunPlus Phoenix,LOUD,2,Round 2,312800,FPX,LOUD,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921
3,"March 17, 2024",Team Heretics,Paper Rex,2,Round 2,312799,TH,PRX,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921
4,"March 18, 2024",LOUD,EDward Gaming,1,Round 3,312803,LOUD,EDG,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921


,Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,...,HS%,CL%,CL Success,CL Attempts,KMax,K,D,A,FK,FD
0,zekken,SEN,64,1.17,289.7,1.28,75.0,185.7,1.00,0.20,...,22.0,40.0,2.0,5.0,27,64,50,13,17,11
1,Zellsis,SEN,64,0.97,184.3,1.02,70.0,124.0,0.69,0.38,...,18.0,22.0,2.0,9.0,21,44,43,24,4,5
2,TenZ,SEN,64,0.96,194.3,0.98,73.0,123.1,0.77,0.44,...,25.0,NaN,0.0,5.0,21,49,50,28,6,5
3,johnqt,SEN,64,0.92,168.7,0.74,66.0,124.1,0.50,0.30,...,19.0,21.0,3.0,14.0,12,32,43,19,2,5
4,Sacy,SEN,64,0.91,171.0,0.73,69.0,113.7,0.55,0.42,...,28.0,14.0,1.0,7.0,17,35,48,27,1,8


### Save basic

In [ ]:
stats_2023.to_csv(folderpath + "stats_2023.csv", index=False)

In [ ]:
# Load from csv
temp_event_stats = pd.read_csv(folderpath + "event_stats.csv")

first_match = temp_event_stats.iloc[2]
display(first_match)
#print(first_match["team1_stats"])
# load first_match["team1_stats"] into a df as csv
temp_df = pd.read_csv(StringIO(first_match["team1_stats"]))
display(temp_df)

date                                               July 22, 2023
team1                                                Team Secret
team2                                                      Gen.G
winner                                                         1
series                                               Lower Final
code                                                      237273
team1_abrev                                                   TS
team2_abrev                                                  GEN
team1_stats    Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...
team2_stats    Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...
Name: 2, dtype: object

,Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,...,HS%,CL%,CL Success,CL Attempts,KMax,K,D,A,FK,FD
0,invy,TS,82,1.22,209.8,1.15,73.0,134.4,0.77,0.52,...,33.0,14.0,1.0,7.0,18,63,55,43,8,3
1,DubsteP,TS,82,1.10,202.0,1.13,65.0,146.1,0.76,0.12,...,28.0,23.0,3.0,13.0,20,62,55,10,10,13
2,Jremy,TS,82,1.10,234.8,1.16,67.0,149.7,0.80,0.20,...,21.0,10.0,1.0,10.0,23,66,57,16,11,14
3,BORKUM,TS,82,1.01,175.8,0.95,74.0,112.3,0.63,0.43,...,23.0,20.0,1.0,5.0,16,52,55,35,6,4
4,JessieVash,TS,82,0.81,156.0,0.64,66.0,109.1,0.46,0.34,...,21.0,17.0,2.0,12.0,12,38,59,28,5,8


# Loading odds data from sqlite database

In [ ]:
#sqllite
import sqlite3
import pandas as pd

files = ['savedataOS.db']

# from the matches table get all the matches, Get the "t1", "t2", "t1oo", "t2oo", and "date_winner"
def get_match_odds_from_sql(files):
  df = pd.DataFrame()
  for file in files:
    conn = sqlite3.connect(file)
    query = "SELECT t1, t2, t1oo, t2oo, date_winner FROM match"
    df = pd.concat([df, pd.read_sql_query(query, conn)])
    conn.close()
  return df
  
match_odds = get_match_odds_from_sql(files)
display(match_odds.head())

,t1,t2,t1oo,t2oo,date_winner
0,ValorsGG,24Haven,1.11,5.02,2022-01-25 19:53:02.215532
1,ex-FLICKBAITERS,Third Impact,1.44,2.48,2022-01-26 01:05:19.012369
2,Built By Gamers Academy,Zero MarksMen Silver,1.2,3.75,2022-01-26 00:56:31.898648
3,Aspyre,Team Reckaroo,1.67,2.0,2022-01-26 01:02:31.354859
4,SoaR,Zero MarksMen Black,1.25,3.35,2022-01-26 00:20:04.412449


## Format date

In [ ]:
# convert date_winner from 2022-01-25 19:53:02.215532 to 2022-01-25
match_odds["date_winner"] = match_odds["date_winner"].str.split(" ").str[0]

# convert date from March 16, 2024 to 2024-03-16
stats_2023["date"] = pd.to_datetime(stats_2023["date"]).dt.strftime('%Y-%m-%d')

display(match_odds.head(1))
display(stats_2023.head(1))

,t1,t2,t1oo,t2oo,date_winner
0,ValorsGG,24Haven,1.11,5.02,2022-01-25


,date,team1,team2,winner,series,code,team1_abrev,team2_abrev,team1_stats,team2_stats,event_code
0,2024-03-16,Sentinels,Karmine Corp,1,Round 2,312798,SEN,KC,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921


## Adds odds to matches from the

In [ ]:
# Add the t1oo and t2oo to the stats_2023 using t1, t2, and date (date can be off by one day give or take)
# not all matches with odds will be in the stats_2023 and vice versa

def Add_odds_to_matches(matches: pd.DataFrame, odds_dfs: pd.DataFrame) -> pd.DataFrame:
  new_matches = matches.copy()
  # add the odds columns
  new_matches["t1 odds"] = np.nan
  new_matches["t2 odds"] = np.nan
  
  # for each match
  for index, match in new_matches.iterrows():
    # get the date (it can be off by one day)
    date = match["date"]
    datep1 = (pd.to_datetime(date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    datem1 = (pd.to_datetime(date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    
    # get the team names
    t1 = match["team1"]
    t2 = match["team2"]

    # find the match in the odds df
    match_ = odds_dfs[(odds_dfs["t1"] == t1) & (odds_dfs["t2"] == t2) & ((odds_dfs["date_winner"] == date) | (odds_dfs["date_winner"] == datep1) | (odds_dfs["date_winner"] == datem1))]
    
    # if the match was found set the odds
    if len(match_) > 0:
      new_matches.at[index, "t1 odds"] = match_["t1oo"].iloc[0]
      new_matches.at[index, "t2 odds"] = match_["t2oo"].iloc[0]
      
  return new_matches

stats_2023_with_odds = Add_odds_to_matches(stats_2023, match_odds)
display(stats_2023_with_odds.head(100))

# print the amount of matches with odds vs without
print("Matches with odds:", len(stats_2023_with_odds[(stats_2023_with_odds["t1 odds"].notnull()) & (stats_2023_with_odds["t2 odds"].notnull())]))
print("Matches without odds:", len(stats_2023))

# print the amount of matches with odds in match_odds
print("Matches with odds in match_odds:", len(match_odds))

,date,team1,team2,winner,series,code,team1_abrev,team2_abrev,team1_stats,team2_stats,event_code,t1 odds,t2 odds
0,2024-03-16,Sentinels,Karmine Corp,1,Round 2,312798,SEN,KC,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.85,1.85
1,2024-03-16,EDward Gaming,Gen.G,2,Round 2,312797,EDG,GEN,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,2.4,1.5
2,2024-03-17,FunPlus Phoenix,LOUD,2,Round 2,312800,FPX,LOUD,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,5.0,1.13
3,2024-03-17,Team Heretics,Paper Rex,2,Round 2,312799,TH,PRX,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.88,1.82
4,2024-03-18,LOUD,EDward Gaming,1,Round 3,312803,LOUD,EDG,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.68,2.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-06-04,Weibo Gaming,Four Angry Men,1,Round 2,221183,WBG,4AM,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1664,NaN,NaN
96,2023-06-04,Rare Atom,Monarch Effect,2,Round 2,221182,RA,ME,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1664,NaN,NaN
97,2023-06-05,Weibo Gaming,Shenzhen NTER,1,Round 3,221837,WBG,NTER,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1664,NaN,NaN
98,2023-06-05,Royal Never Give Up,Nova Esports,1,Round 3,221838,RNG,NOVA,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1664,NaN,NaN


Matches with odds: 203
Matches without odds: 376
Matches with odds in match_odds: 888


In [ ]:
#save to csv
stats_2023_with_odds.to_csv(folderpath + "stats_2023_odds.csv", index=False)


In [ ]:
# from csv
stats_2023_with_odds = pd.read_csv(folderpath + "stats_2023_odds.csv")

In [ ]:
def get_match_link(match_code) -> str:
  return f"https://www.vlr.gg/{match_code}"


## Add hed to head and previous match result

In [ ]:
def get_head_to_head_score(match_row: pd.Series, soup: BeautifulSoup) -> tuple[int, int]:
  # scores are labeled with class match-h2h-matches-score
  head_t_heads = soup.find_all("div", class_="match-h2h-matches-score")
  head_t_head_dates = soup.find_all("div", class_="match-h2h-matches-date")

  round_differential = []
  for head_t_head in head_t_heads:
    # scores are in the two sub divs
    scores_divs = head_t_head.find_all("span")
    
    t1_score = int(scores_divs[0].text.strip())
    t2_score = int(scores_divs[1].text.strip())
    
    round_differential.append(t1_score - t2_score)

  date = pd.to_datetime(match_row["date"])

  months = []
  for date_div in head_t_head_dates:
    # change from YYYY/MM/DD to YYYY-MM-DD
    head_t_head_date = pd.to_datetime(date_div.text.replace("/", "-"))
    # get the months since the match
    months.append(12-((date - head_t_head_date).days / 30))

  # calculate the head to head score for the two teams (win * months since row["date"])
  t1_score = 0
  t2_score = 0

  for i in range(len(months)):
    # skip if months is negative
    if months[i] <= 0:
      continue
    
    if round_differential[i] > 0:
      t1_score += round_differential[i] * months[i]
    elif round_differential[i] < 0:
      t2_score += abs(round_differential[i]) * months[i]
      
  return t1_score, t2_score

def get_previous_matches_score(soup: BeautifulSoup) -> tuple[int, int]:
  # find the block with a stype of "display: flex;"
  prev_block = soup.find("div", style="display: flex;")
  
  # get the previous matches in div with class that starts with "match-histories-item"
  previous_matches = prev_block.find_all("a", class_=lambda x: x and x.startswith("match-histories-item"))
  
  game_scores = []
  for match in previous_matches:
    # if the class contains "mod-win"
    if "mod-win" in match["class"]:
      game_scores.append(True)
    else:
      game_scores.append(False)
      
  # first 5 are team 1, last 5 are team 2
  team1_score = sum(game_scores[:5])
  team2_score = sum(game_scores[5:])
  return team1_score, team2_score

def add_previous_games_scores(data: pd.DataFrame) -> pd.DataFrame:
  new_data = data.copy()
  new_data["team1_previous_score"] = 0
  new_data["team2_previous_score"] = 0
  
  new_data["team1_hth"] = 0
  new_data["team2_hth"] = 0
  
  for index, row in new_data.iterrows():
    print(index, end="\r")
    code = row["code"]
    soup = BeautifulSoup(requests.get(get_match_link(code)).content, 'html.parser')
    t1_score, t2_score = get_previous_matches_score(soup)
    new_data.at[index, "team1_previous_score"] = t1_score
    new_data.at[index, "team2_previous_score"] = t2_score
    
    t1_hth, t2_hth = get_head_to_head_score(row, soup)
    new_data.at[index, "team1_hth"] = t1_hth
    new_data.at[index, "team2_hth"] = t2_hth
    
  return new_data

In [ ]:
final_stats = add_previous_games_scores(stats_2023_with_odds)

# [Link to match stats](https://www.vlr.gg/314634/sentinels-vs-mibr-champions-tour-2024-americas-stage-1-w3)

In [ ]:
display(final_stats.head(10))

,date,team1,team2,winner,series,code,team1_abrev,team2_abrev,team1_stats,team2_stats,event_code,t1 odds,t2 odds,team1_previous_score,team2_previous_score,team1_hth,team2_hth
0,2024-03-16,Sentinels,Karmine Corp,1,Round 2,312798,SEN,KC,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.85,1.85,4,5,0.000000,0.000000
1,2024-03-16,EDward Gaming,Gen.G,2,Round 2,312797,EDG,GEN,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,2.40,1.50,4,5,0.000000,0.000000
2,2024-03-17,FunPlus Phoenix,LOUD,2,Round 2,312800,FPX,LOUD,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,5.00,1.13,3,3,0.000000,0.000000
3,2024-03-17,Team Heretics,Paper Rex,2,Round 2,312799,TH,PRX,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.88,1.82,3,2,0.000000,0.000000
4,2024-03-18,LOUD,EDward Gaming,1,Round 3,312803,LOUD,EDG,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.68,2.05,3,4,4.966667,5.666667
5,2024-03-18,Karmine Corp,Paper Rex,2,Round 3,312802,KC,PRX,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.85,1.85,4,3,0.000000,0.000000
6,2024-03-21,Gen.G,Paper Rex,1,Upper Semifinals,312777,GEN,PRX,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.85,1.85,5,3,22.333333,14.000000
7,2024-03-21,Sentinels,LOUD,1,Upper Semifinals,312776,SEN,LOUD,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,1.78,1.92,4,3,11.400000,22.633333
8,2024-03-22,Gen.G,Sentinels,1,Upper Final,312778,GEN,SEN,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,2.10,1.65,5,5,0.000000,0.000000
9,2024-03-22,Paper Rex,LOUD,1,Lower Round 1,312780,PRX,LOUD,"Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...","Player,Team,Rnd,R,ACS,K:D,KAST,ADR,KPR,APR,FKP...",1921,2.70,1.42,2,2,4.800000,0.000000


# Save final DB

In [ ]:
# save to csv
final_stats.to_csv(folderpath + "final_stats.csv", index=False)